# Iterative Solvers for Elliptic PDEs

In "Intro to Finite Difference Methods for PDEs" we classified the $2^{nd}$ order quasilinear PDEs as elliptic, parabolic, or hyperbolic, and we started looking as solvers for elliptic equations.

In particular, we looked at the 2D Laplace equation:
- 2D Cartesian (rectangular, coord-aligned) domain (coords: $\{x,y\}$)
- Dirichlet BCs (values specified on boundary)

$$ \nabla^2 u = \frac{\partial^2u}{\partial x^2} + \frac{\partial^2u}{\partial y^2} = 0 \text{ on } x \in [0,L_x]; \; y \in [0,L_y]$$

BCs: $u(0,y)=u(L_x,y)=u(0,x)=0, \; u(L_y,x) = sin(\pi x)$

After discretizing the spatial domain to a regular 2D grid, we discretized the PDE by substituting with central difference approximations for the derivatives:

$$
\begin{aligned}
\frac{\partial^2u}{\partial x^2} & \rightarrow \frac{1}{h^2}(u_{i-1,j}-2u_{i,j}+u_{i+1,j}) + O(h^2) \\
\frac{\partial^2u}{\partial y^2} & \rightarrow \frac{1}{h^2} (u_{i,j-1} -2 u_{i,j} + u_{i,j+1}) + O(h^2)
\end{aligned}
$$
- For each internal grid point get equation of the form:

$$ \nabla^2 u = \frac{\partial ^2 u}{\partial x^2} + \frac{\partial^2u}{\partial y^2} \rightarrow u_{i-1,j}+u_{i+1,j}+u_{i,j-1} + u_{i,j+1} -4u_{i,j} = 0$$

or in code format:

`u[i-1,j] + u[i+1,j] + u[i,j-1] + u[i,j+1] - 4*u[i,j] == 0`

$\implies$ __5-point stencil computation__: 

![stencil_5point](stencil_5point.png)

With no source terms (so right-hand side is zero), we can solve for $u_{i,j}$ in terms of values at neighboring gridpoints. At equilibrium (for the actual solution for the steady-state temperature distribution), the following relation should satisfied (in the limit as the grid is refined):

$$u_{i,j} = \frac{1}{4} (u_{i-1,j} + u_{i+1,j} +u_{i,j-1} +u_{i,j+1} )$$

We turn this into an iterative scheme by adding an iteration count superscript, so that at each iteration the value at each grid point is replaced by the stencil based on the neighbors (in this case, by the average of teh neighboring values).

$$u^{k+1}_{i,j} = \frac{1}{4} (u^k_{i-1,j} + u^k_{i+1,j} +u^k_{i,j-1} +u^k_{i,j+1} )$$

Game plan for computing solution:
  - Repeatedly update each grid point. 
  - Converge to numerical solution!?
  - This approach is known as Jacobi iteration.

Let's take a look at implementation and convergence issues:

While truncation error is $O(h^2)$, refining grid may not reduce error.

Also need to worry about rate of convergence:
- Jacobi spectral radius (magnitude of largest eigenvalue of the BIG matrix): 
$$\rho_{jacobi} \approx 1-\frac{1}{2} (\pi^2 h^2)$$
- Good news: $\rho_{jacobi} < 1 \implies$ convergence
- Bad news: grid refinement $\implies h \to 0, \rho_{jacobi} \to 1$
- Attempts to reduce truncation error by refinement cause SLOW convergence
- Number of iterations for given accuracy $\sim O(h^2)$
- Total operation count $\sim O(N^4) \implies$ Serial = Painful
- Sub-optimal memory usage: <br>new values computed from old values $\iff$ need a copy of the array  

Alternatives:
- Gauss-Seidel: systematically use updated (improved) values
- Can operate "in place" with single version of array
- Spectral radius: 
$$\rho_{GS} \approx 1-(\pi^2 h^2)$$ 
    - Improved
    - But only by factor of 2
    - CUDA parallel version will be non-deterministic
- Red-Black version (inspired by checkerboard)
  - Grid points of one color updated bsed only on other color
  - Run kernel twice:
    - update red points
    - update black points
  - Deterministic and has improved Gauss-Seidel convergence rate
- Successive over-relaxation
  - Think of the change in values of $u$ as a direction to move to reduce the residual error

$$\Delta u_{i,j} = u^{k+1}_{i,j}-u^k_{i,j} = \frac{1}{4} (u^k_{i-1,j} + u^k_{i+1,j} +u^k_{i,j-1} +u^k_{i,j+1} -4u^k_{i,j})$$

```
du = (u[i-1,j]+u[i+1,j]+u[i,j-1]+u[i,j+1]-4.*u[i,j])/4.
```

  - If it is good to move in that direction, wouldn't it be better to mover farther in that direction?

$$u^{k+1}_{i,j} = u^k_{i,j} + w * \Delta u_{i,j}$$
```
u[i,j] = u[i,j] + w * du
```
  - `w` is the over-relaxation factor

  - Don't get carried away: $w>2$ $\implies$ divergence
  - Optimal value: $w_{opt} = 2-2 \pi h$
  - Optimal convergence: 
  $$\rho_{opt} = 1 - 2 \pi h $$
  - As grid is refined, convergence multiplier approaches 1 more slowly; iterations required for given accuracy closer to $O(N)$

### Improved stencils

Another way to strive for improved performance of the solver is to use stencils with higher order truncation error. The relatively obvious approach to this is to employ a tensor product stencil with a larger radius. For example, one could use the second derivative finite difference operator with radius 2:

$$ \frac{\partial^2 u_{i,j}}{\partial x^2} = \frac{1}{12 h^2} (-u_{i-2,j}+16 u_{i-1,j}-30u_{i,j}+16 u_{i+1,j}-u_{i+2,j}) + O(h^4)$$

and the corresponding expression for $\frac{\partial^2 u_{i,j}}{\partial y^2}$ to produce a 9-point stencil of radius 2 with $4^{th}$-order truncation error. This would lead to the possibility of improved accuracy, but at the cost of slightly more than doubling the required number of halo values.

However, there is another version of a 9-point Laplacian stencil to consider. This stencil has radius 1 but includes the central point along with all 8 of its neighbors including along the diagonals of the grid. The corresponding approximation formula is:

$$∇^2
_9u_{i,j} = \frac{1}{6 h^2}
[u_{i−1,j−1} + 4u_{i−1,j} + 4 u_{i+1,j} + u_{i+1,j+1}
−20u_{i,j} + u_{i+1,j−1} + 4u_{i,j−1} + 4u_{i,j+1} + u_{i−1,j+1}] $$

so the stencil coefficients are $-1$ at the center, $1/4$ at the edge, and $1/20$ at the corner (multipled by a constanct factor $\frac{20}{6h^2}$).

The relation to the exact Laplace operator is:

$$∇^2_9 u_(x_i, y_i) = ∇^2u + \frac{h^2}{2}[u_{xxxx} + 2u_{xxyy} + u_{yyyy}] + O(h^4)$$

which appears to also have second-order truncation error. However, the leading order error term turns out to involve the Laplacian of the Laplacian (also know as the ___biharmonic operator___ which you may have run into previously when studying elasticity), and we can rewrite the expression as:

$$∇^2_9 u(x_i, y_i) = ∇^2u + \frac{h^2}{2} \nabla^2(\nabla^2 u) + O(h^4)$$

and take advantage of this relationship to improve solution accuracy. Substituting the 9-point Laplacian stencil into the Poisson equation, $\nabla^2u = F$, gives:

$$∇^2_9 u(x_i, y_i) = F + \frac{h^2}{2} \nabla^2F + O(h^4)$$

If the forcing term $F$ is sufficiently smooth (twice-differentiable), we can intentially introduce the "error" $\frac{h^2}{2} \nabla^2F$ into the forcing term, and solve with the 9-point stencil to obtain a solution with $4^{th}$-order truncation error. Laplace's equation corresponds to the case when $F=0$, so $\nabla^2F=0$ and the leading error term automatically vanishes and the 9-point stencil produces a $4^{th}$-order accurate solution without needing to introduce a fictitious error term.

Our interest lies in parallelizing these methods and characterizing their performance

- Compared to CPU world:
  - less concerned about operation counts
  - more concerned about memory and iterations (when forced to serialize)
  - 2D tiled approach using shared memory with halo values

Let's look at sample serial and parallel codes...

The listing below implements iterative solutions (including Jacobi iteration, Gauss-Seidel iteration, and Successive Over-relaxation) for the sample Laplace equation problem presented earlier in the notebook. Be selecting a method, number of grid points, and number of iterations, several salient results can be demonstrated:

- Each of the iterations does converge toward the exact solution.
- The rate of convergence decreases as the grid is refined by increasing the number of grid points.
- Refining the grid does not suffice to decrease the error; the number of iterations must also be increased.
- To achieve similar errors, it takes about 2 Jacobi updates for each Gauss-Seidel update.
- Successive over-relaxation provides a significant improvement in the convergence rate.
  
```
File: jacobi_serial.py
01: # #%%
02: import numpy as np
03: import matplotlib.pyplot as plt
04: from math import sin, sinh
05: 
06: NX, NY = 11,11
07: iters = 1 #NX*NX
08: PI = np.pi
09: method = 'Jacobi' #choices: 'Jacobi','Gauss-Seidel', 'SOR' 
10: 
11: def mean_update(u, method):
12:     '''
13:     update 2D array with non-boundary elements replaced by average of 4 nearest neighbors (on Cartesian grid)
14:     Args:
15:         u: 2D numpy array of floats
16:         method: string specifying 'Jacobi', 'Gauss-Seidel' or 'SOR'
17:     Returns:
18:         updated numpy array with same shape as u
19:     '''
20:     nx, ny = np.shape(u)
21: 
22:     if method == 'Jacobi':
23:         u_new = np.copy(u)
24:         for i in range(1,nx-1):
25:             for j in range(1,ny-1):
26:                 u_new[i,j] = (u[i-1,j]+u[i+1,j]+u[i,j-1]+u[i,j+1])/4.
27:         return u_new
28: 
29:     if method == 'Gauss-Seidel':
30:         for i in range(1,nx-1):
31:             for j in range(1,ny-1):
32:                 u[i,j] = (u[i-1,j]+u[i+1,j]+u[i,j-1]+u[i,j+1])/4.
33:         return u
34: 
35:     if method == 'SOR':
36:         h = 1./(nx-1)
37:         w = 2. * (1-2*PI*h)
38:         for i in range(1,nx-1):
39:             for j in range(1,ny-1):
40:                 u[i,j] =  u[i,j] + w*(u[i-1,j]+u[i+1,j]+u[i,j-1]+u[i,j+1]-4.*u[i,j])/4.
41:         return u
42: 
43: def main():
44:     #Compute exact solution
45:     exact = np.zeros(shape=[NX,NY], dtype=np.float32)
46:     for i in range(NX):
47:         for j in range(NY):
48:             exact[i,j]= sin(i*PI/(NX-1)) * sinh(j*PI/(NY-1))/sinh(PI)
49:     
50:     #serial iteration results
51:     u = np.zeros(shape=[NX,NY], dtype=np.float32)
52:     for i in range(NX):
53:         u[i,NX-1]= sin(i*PI/(NX-1))
54:     for k in range(iters):
55:         u = mean_update(u, method)
56:     
57:     error = np.max(np.abs(u-exact))
58:     print("%s, NX = %d, iters = %d => max error: %5.2e"  %(method, NX, iters, error))
59: 
60:     xvals = np.linspace(0., 1.0, NX)
61:     yvals = np.linspace(0., 1.0, NY)
62:     X,Y = np.meshgrid(xvals, yvals)
63:     levels = [0.025, 0.1, 0.25, 0.50, 0.75]
64:     plt.contourf(X,Y,exact.T, levels = levels)
65:     plt.contour(X,Y,u.T, levels = levels,
66:         colors = 'r', linewidths = 4)
67:     plt.axis([0,1,0,1])
68:     plt.show()
69: 
70: if __name__ == '__main__':
71:     main()
72:
```

Now let's move on to look at a parallel implementation of Jacobi iteration. Salient features to note include:

- The code implements a "tiled" approach reading a block of data (plus halo values) into a shared memory array where it can be accessed efficiently.
- The parallel code only implements Jacobi iteration but, in addition to the 5-point stencil, it also includes the 9-point stencil of radius 1 that provides $4^{th}$-order accuracy.
- Two device arrays, `d_u` and `d_v`, are created and they take turns serving as input and output arrays.
- The results are not copied back to the host at every step, so `copy_to_host()` is called only once at the very end of the wrapper function `update()`.


## Lines 18-55 of the listing below provide a template you can use in your implementation of other 2D tiled methods.
```
File: jacobi_parallel.py
001: # #%%
002: import numpy as np
003: import matplotlib.pyplot as plt
004: from math import sin, sinh
005: from numba import jit, cuda, float32, int32
006: 
007: NX, NY = 21, 21
008: iters = NX*NX//2
009: PI = np.pi
010: STENCIL_POINTS = 9 #specify 5 or 9 for points in stencil
011: TPB = 8
012: RAD = 1
013: SH_N = 10
014: 
015: #kernel with 2D shared memory array including halo
016: @cuda.jit
017: def updateKernel(d_v, d_u, edge, corner):
018:     i,j = cuda.grid(2)
019:     dims = d_u.shape
020:     if i >= dims[0] or j >= dims[1]:
021:         return
022:     NX, NY = cuda.blockDim.x, cuda.blockDim.y
023:     t_i, t_j = cuda.threadIdx.x, cuda.threadIdx.y
024:     sh_i, sh_j = t_i + RAD, t_j + RAD
025: 
026:     sh_u = cuda.shared.array(shape = (SH_N,SH_N), dtype = float32)
027: 
028:     #Load regular values
029:     sh_u[sh_i, sh_j] = d_u[i, j]
030:     
031:     #Halo edge values
032:     if t_i<RAD:
033:         sh_u[sh_i - RAD, sh_j] = d_u[i-RAD, j]
034:         sh_u[sh_i + NX , sh_j] = d_u[i+NX , j]
035: 
036:     if t_j<RAD:
037:         sh_u[sh_i, sh_j - RAD] = d_u[i, j - RAD]
038:         sh_u[sh_i, sh_j + NY ] = d_u[i, j + NY ]
039: 
040:     #Halo corner values
041:     if t_i<RAD and t_j<RAD:
042:         #upper left
043:         sh_u[sh_i - RAD, sh_j - RAD] = d_u[i-RAD, j - RAD]
044:         sh_u[sh_i - RAD, sh_j - RAD] = d_u[i-RAD, j - RAD]
045:         #upper right
046:         sh_u[sh_i + NX, sh_j - RAD] = d_u[i + NX, j - RAD]
047:         sh_u[sh_i + NX, sh_j - RAD] = d_u[i + NX, j - RAD]
048:         #lower left
049:         sh_u[sh_i - RAD, sh_j + NY] = d_u[i-RAD, j + NY]
050:         sh_u[sh_i - RAD, sh_j + NY] = d_u[i-RAD, j + NY]
051:         #lower right
052:         sh_u[sh_i + NX, sh_j + NX] = d_u[i + NX, j + NY]
053:         sh_u[sh_i + NX, sh_j + NY] = d_u[i + NX, j + NY]
054: 
055:     cuda.syncthreads()
056: 
057:     if i>0 and j>0 and i<dims[0]-1 and j<dims[1]-1:
058:         d_v[i, j] = \
059:             sh_u[sh_i-1, sh_j -1]*corner + \
060:             sh_u[sh_i, sh_j -1]*edge + \
061:             sh_u[sh_i+1, sh_j -1]*corner + \
062:             sh_u[sh_i-1, sh_j]*edge + \
063:             sh_u[sh_i, sh_j]*0. + \
064:             sh_u[sh_i+1, sh_j]*edge + \
065:             sh_u[sh_i-1, sh_j +1]*corner + \
066:             sh_u[sh_i, sh_j + 1] * edge + \
067:             sh_u[sh_i+1, sh_j +1]*corner
068:             # edge * (sh_u[sh_i-1, sh_j] + sh_u[sh_i+1, sh_j] + \
069:             #     sh_u[sh_i, sh_j-1] + sh_u[sh_i, sh_j+1]) + \
070:             # corner * (sh_u[sh_i-1, sh_j -1] + sh_u[sh_i+1, sh_j +1] + \
071:             #     sh_u[sh_i-1, sh_j +1] + sh_u[sh_i+1, sh_j-1])
072: 
073: def update(u, iter_count, edge, corner):
074:     d_u = cuda.to_device(u)
075:     d_v = cuda.to_device(u)
076:     dims = u.shape
077:     gridSize = [(dims[0]+TPB-1)//TPB, (dims[1]+TPB-1)//TPB]
078:     blockSize = [TPB, TPB]
079: 
080:     for k in range(iter_count):
081:         updateKernel[gridSize, blockSize](d_v, d_u, edge, corner)
082:         updateKernel[gridSize, blockSize](d_u, d_v, edge, corner)
083: 
084:     return d_u.copy_to_host()
085: 
086: 
087: def main():
088:     #Compute exact solution
089:     exact = np.zeros(shape=[NX,NY], dtype=np.float32)
090:     for i in range(NX):
091:         for j in range(NY):
092:             exact[i,j]= sin(i*PI/(NX-1)) * sinh(j*PI/(NY-1))/sinh(PI)
093: 
094:     #parallel iteration results
095:     u = np.zeros(shape=[NX,NY], dtype=np.float32)
096:     for i in range(NX):
097:         u[i,NX-1]= sin(i*PI/(NX-1)) #boundary conditions
098:     if STENCIL_POINTS == 5:
099:         edge, corner = 0.25, 0
100:     elif STENCIL_POINTS == 9:
101:         edge, corner = 0.20, 0.05
102:     else:
103:         print("Supported values of STENCIL_POINTS: {5,9}")
104:         return
105:     u = update(u, iters, edge, corner)
106: 
107:     error = np.max(np.abs(u-exact))
108:     print("NX = %d, iters = %d => max error: %5.2e"  %(NX, iters, error))
109: 
110:     xvals = np.linspace(0., 1.0, NX)
111:     yvals = np.linspace(0., 1.0, NY)
112:     X,Y = np.meshgrid(xvals, yvals)
113:     levels = [0.025, 0.1, 0.25, 0.50, 0.75]
114:     plt.contourf(X,Y,exact.T, levels = levels)
115:     plt.contour(X,Y,u.T, levels = levels,
116:         colors = 'r', linewidths = 4)
117:     plt.axis([0,1,0,1])
118:     plt.show()
119: 
120: if __name__ == '__main__':
121:     main()
122: 
```